# Feasible mode enumeration vs Reachability cuts

In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
from pympc.dynamics.discrete_time_systems import LinearSystem, AffineSystem, PieceWiseAffineSystem
from pympc.plot import plot_state_space_trajectory
from pympc.geometry.polyhedron import Polyhedron

In [3]:
m = 1.
l = 1.
g = 10.
k = 100.
d = .1
h = .01

In [4]:
# discretization method
method = 'explicit_euler'

# dynamics n.1 (free space)
A1 = np.array([[0., 1.],[g/l, 0.]])
B1 = np.array([[0.],[1/(m*l**2.)]])
S1 = LinearSystem.from_continuous(A1, B1, h, method)

# dynamics n.2 (right wall)
A2 = np.array([[0., 1.],[g/l-k/m, 0.]])
B2 = B1
c2 = np.array([0., k*d/(m*l)])
S2 = AffineSystem.from_continuous(A2, B2, c2, h, method)

# dynamics n.2 (left wall)
A3 = np.array([[0., 1.],[g/l-k/m, 0.]])
B3 = B1
c3 = np.array([0., -k*d/(m*l)])
S3 = AffineSystem.from_continuous(A3, B3, c3, h, method)

# list of dynamics
S = [S1, S2, S3]

In [5]:
# state domain n.1
x1_min = np.array([-d/l, -1.])
x1_max = np.array([d/l, 1.])
X1 = Polyhedron.from_bounds(x1_min, x1_max)
assert X1.bounded

# state domain n.2
x2_min = np.array([d/l, -1.])
x2_max = np.array([2.*d/l, 1.])
X2 = Polyhedron.from_bounds(x2_min, x2_max)
assert X2.bounded

# state domain n.3
x3_min = np.array([-2.*d/l, -1.])
x3_max = np.array([-d/l, 1.])
X3 = Polyhedron.from_bounds(x3_min, x3_max)
assert X3.bounded

# input domain
u_min = np.array([-4.])
u_max = np.array([4.])
U = Polyhedron.from_bounds(u_min, u_max)
assert U.bounded

# domains
D = [Xi.cartesian_product(U) for Xi in [X1, X2, X3]]

In [6]:
PWA = PieceWiseAffineSystem(S, D)

def dynamics(x1, x2):
    x = np.array([x1, x2])
    return PWA.simulate(x, [np.zeros(PWA.nu)])[0][1]
fig = plt.figure()
n = 21
x1 = np.linspace(-2.*d/l, 2.*d/l, n)
x2 = np.linspace(-1., 1., n)
X1, X2 = np.meshgrid(x1, x2)
F1 = np.array([dynamics(i, j)[0] - i for i, j in zip(np.ravel(X1), np.ravel(X2))]).reshape(n, n)
F2 = np.array([dynamics(i, j)[1] - j for i, j in zip(np.ravel(X1), np.ravel(X2))]).reshape(n, n)
M = np.hypot(F1, F2)
Q = plt.quiver(X1, X2, F1, F2, M)
# plt.axis('equal')
# plt.xlabel(r'$x_1$')
# plt.ylabel(r'$x_2$')
x0 = np.array([-1., 0.])
# x_traj, ms_traj = PWA.simulate(x0, [np.zeros(2)]*100)
# plot_state_space_trajectory(x_traj)
plt.show()

In [7]:
from pympc.control.hybrid_benchmark.utils import infeasible_mode_sequences
infeasible_mode_sequences(PWA,5)

[(1, 2),
 (2, 1),
 (1, 0, 2),
 (2, 0, 1),
 (1, 0, 0, 2),
 (2, 0, 0, 1),
 (1, 0, 0, 0, 2),
 (2, 0, 0, 0, 1)]

In [8]:
from pympc.control.hybrid_benchmark.controllers import HybridModelPredictiveController
from pympc.control.hybrid_benchmark.utils import (get_constraint_set,
                                                  remove_redundant_inequalities_fast,
                                                  convex_hull_method_fast,
                                                  infeasible_mode_sequences)

In [9]:
# controller parameters
N = 30
Q = np.diag([1.,1.])
R = np.diag([1.])

# terminal set and cost
P, K = S1.solve_dare(Q, R)
X_N = S1.mcais(K, D[0])

In [10]:
controller = HybridModelPredictiveController(PWA, N, Q, R, P, X_N, method='Convex hull, lifted constraints')

In [11]:
from pympc.control.hybrid_benchmark.branch_and_bound import branch_and_bound, best_first, depth_first, first_depth_then_breadth, first_depth_then_best

In [13]:
x0 = np.array([.09,.1])
# sol = controller.feedforward(x0)

In [14]:
def solver(identifier, objective_cutoff):
    x0 = np.array([.09,.7])
    return controller.solve_relaxation(x0, identifier, objective_cutoff)

In [29]:
sol, lb, ub = branch_and_bound(solver, first_depth_then_breadth, controller.explore_in_chronological_order)

|     Updates    |     Time (s)   |    Nodes (#)   |   Lower bound  |   Upper bound  |
 ---------------- ---------------- ---------------- ---------------- ----------------
  Root node        0.020            1                2.713            inf             
  New incumbent    1.192            34               4.792            107.482         
  Solution found   4.576            80               107.482          107.482         

Explored 80 nodes in 4.576 seconds: solution found with objective 107.482.
Best lower bound is 107.482 and lies within the tolerance of 0.000.


In [ ]:
np.save(method + '_lb', lb)
np.save(method + '_ub', ub)

In [47]:
lb = {}
ub = {}
for method in methods:
    lb[method] = np.load(method + '_lb.npy')
    ub[method] = np.load(method + '_ub.npy')

mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')
plt.rc('font', size=14)
colors = ['b', 'r', 'c', 'g']
linestyles = ['-','-.','--',':']
for i, method in enumerate(methods):
    plt.plot(lb[method], label=method, color=colors[i], linestyle=linestyles[i], linewidth=3)
    plt.plot(ub[method], label=method, color=colors[i], linestyle=linestyles[i], linewidth=3)

plt.plot(lb/sol['objective'])
plt.plot(ub/sol['objective'])
plt.xlim((0, len(lb)))


# plt.ylim((0, 1.1*max([b for b in ub if not np.isinf(b)])/sol['objective']))
plt.legend()
plt.grid(True)
plt.xlabel(r'Node in B&B algorithm')
plt.ylabel(r'Lower and upper bound / cost MICP')
# plt.savefig('relaxation_ratio.pdf', bbox_inches='tight')
plt.show()

NameError: name 'methods' is not defined

In [ ]:
X1.plot(label=r'$\mathcal{X}_1$', facecolor='g')
X2.plot(label=r'$\mathcal{X}_2$', facecolor='b')
X_N.plot(label=r'$\mathcal{X}_N$', facecolor='r')
plot_state_space_trajectory(sol['state'], label='open loop', color='c')
plt.legend()
plt.show()

In [ ]:
x0 = np.array([.1,.3])
controller = HybridModelPredictiveController(PWA, N, Q, R, P, X_N, method='Convex hull')
# controller.add_reachability_constraints(5)
# u,x,ms,cost = controller.feedforward(x0)
u,x,ms,cost,d = controller.feedforward_relaxation(x0,[])
print cost
ims = infeasible_mode_sequences(PWA, 5)
i = 8
print ims[i]
for t in range(N-len(ims)):
    if sum([d[t+tau][m] for tau, m in enumerate(ims[i])]) >= len(ims[i]) - 1.:
        print 'error'
#     for tau, m in enumerate(ims[i]):
#         print t, tau, m, d[t+tau][m]
#     print 'sum:', sum([d[t+tau][m] for tau, m in enumerate(ims[i])]) <= len(ims[i] - 1.)
#     print '\n'

In [ ]:
from pympc.plot import plot_state_space_trajectory

In [ ]:
X_N.plot()
plot_state_space_trajectory(x)

In [ ]:
x_min = np.minimum.reduce([x1_min, x2_min, x3_min])
x_max = np.maximum.reduce([x1_max, x2_max, x3_max])

In [ ]:
# samples for the initial position and velocity of the ball
n_levels = 10
n_samples = 101
q_samples = np.linspace(x_min[0], x_max[0], n_samples)
v_samples = np.linspace(x_min[1], x_max[1], n_samples)
Q_samples, V_samples = np.meshgrid(q_samples, v_samples)

In [ ]:
from pympc.control.hybrid_benchmark.utils import get_constraint_set, remove_redundant_inequalities_fast, convex_hull_method_fast

# get feasible sets (uses the feasible set of the CH method for the CHLC method)
controller = HybridModelPredictiveController(PWA, N, Q, R, P, X_N, method='Convex hull')
# controller.add_reachability_constraints(10)
cs = get_constraint_set(controller.prog)
var_indices = {v: i for i, v in enumerate(controller.prog.getVars())}
assert var_indices[controller.prog.getVarByName('x0[0]')] == 0
assert var_indices[controller.prog.getVarByName('x0[1]')] == 1
sect = convex_hull_method_fast(cs, [0,1])

# samples on the grid
cost_mat = np.empty([n_samples]*2)
for i, q in enumerate(q_samples):
    for j, v in enumerate(v_samples):
        print(str(i) + ',' + str(j) + '   \r'),
        x0 = np.array([q, v])
        cost = controller.feedforward_relaxation(x0, [])[3]
        cost_mat[i,j] = cost

In [ ]:
from math import floor
def my_round(x):
    return floor(x*10.)/10.

levels = [my_round((i+1)*np.nanmax(cost_mat)/n_levels) for i in range(n_levels)]
cp = plt.contour(Q_samples, V_samples, cost_mat.T, levels=levels, cmap='viridis_r')
plt.colorbar(cp, label='Cost relaxed problem')
sect.plot(facecolor='w')
plt.grid(True)
plt.savefig('pendulum_2_walls_bm.pdf',bbox_inches='tight')
plt.show()